<a href="https://colab.research.google.com/github/CEZERT/Gpkg_Postgis/blob/main/Jamb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!import fiona
!import geopandas
!import psycopg2

In [ ]:
import fiona
import psycopg2
import geopandas as gpd

# Define GeoPackage file path and PostGIS connection parameters
gpkg_path = r"S:\09-Production\CLIENTS\AIR LIQUIDE\Suivi_jambonnage\14.QFIELD\2023\NF\projet_QGIS\RETOUR TERRAIN\Tab_01\data.gpkg"
pg_user = "postgres"
pg_password = "postgres"
pg_host = "172.16.0.28"
pg_port = "5432"
pg_database = "jamb_airl_qfield_bis"

# Connect to PostGIS database
pg_conn = psycopg2.connect(
    user=pg_user,
    password=pg_password,
    host=pg_host,
    port=pg_port,
    database=pg_database)

# Read GeoPackage file into dictionary of GeoDataFrames
gpkg_gdfs = gpd.read_file(gpkg_path)

# Loop through layers in GeoDataFrames
for layer_name, layer_gdf in gpkg_gdfs.items():

    layer_gdf = layer_gdf.convert_dtypes()
    layer_fields = layer_gdf.dtypes.to_dict()
    layer_field_names = list(layer_fields.keys())
    layer_field_types = list(layer_fields.values())

    # Check if layer exists in PostGIS database
    pg_cursor = pg_conn.cursor()
    pg_cursor.execute(
        f"SELECT EXISTS (SELECT 1 FROM information_schema.tables WHERE table_schema = 'public' AND table_name = '{layer_name}')")
    layer_exists = pg_cursor.fetchone()[0]

    if layer_exists:
        # Compare fields between GeoPackage layer and PostGIS table
        pg_cursor.execute(
            f"SELECT column_name, data_type FROM information_schema.columns WHERE table_name = '{layer_name}'")
        pg_fields = pg_cursor.fetchall()
        pg_field_names = [field[0] for field in pg_fields]
        pg_field_types = [field[1] for field in pg_fields]

        if layer_field_names != pg_field_names or layer_field_types != pg_field_types:
            print(f"Fields in layer {layer_name} do not match fields in table {layer_name} in PostGIS database.")
    else:
        print(f"Layer {layer_name} does not exist in PostGIS database.")

# Close PostGIS connection
pg_cursor.close()
pg_conn.close()()